In the same directory, there is `talents.txt` file. That is PDF content provided by Gallup(tm) please note that they are owner of this content.

Countries taken from here: https://gist.github.com/pamelafox/986163 - I had to do some changes, that is why I have a copy.

To run this script pandas, folium, xlrd, xlwt modules are needed.

Have a fun!

Take `talents.txt`, and creates `talents2.xls` with geo coordinates. 

In [ ]:
from helper import convert_to_excel

#convert_to_excel('talents.txt', 'talents2.xls')

Read dataframe from `talents2.xls`

In [ ]:
import pandas as pd

df = pd.read_excel('talents2.xls')

remove females, males, all entries (the ones without coordinates)

In [ ]:
df = df.dropna(subset=['lon'])
df = df.dropna(subset=['lat'])

Print out map with Learners!

In [ ]:
import folium
from folium import plugins


def put_piechart(folium_map, location, row):
    popup_text = f"country: {row['country']}<br> respondants: {row['respondents']}"
    if row['respondents'] and row['population']:
        cover =  10000000.0 * float(row['respondents']) / float(row['population'])
    else:
        cover = 1
    if row['respondents']:
        respondents = float(row['respondents']) / 10.0
    else:
        respondents = 1
    if row['population']:
        population = float(row['population']) / 1000.0
    else:
        population = 1

    plugins.SemiCircle(location=(row['lat'], row['lon']), radius=cover, start_angle=0, stop_angle=120,
               color="#007849", fill=True, fill_color="#015627", fill_opacity=0.7,
               popup=popup_text, tooltip=f"{row['country']}"
              ).add_to(folium_map)
    plugins.SemiCircle(location=(row["lat"], row["lon"]), radius=respondents, start_angle=120, stop_angle=240,
               color="#8BB0EC", fill=True, fill_color="#6990BA", fill_opacity=0.7,
               popup=popup_text, tooltip=f"{row['country']}"
              ).add_to(folium_map)
    plugins.SemiCircle(location=(row["lat"], row["lon"]), radius=population, start_angle=240, stop_angle=360,
               color="#FF3C3C", fill=True, fill_color="#BB0909", fill_opacity=0.7,
               popup=popup_text, tooltip=f"{row['country']}"
              ).add_to(folium_map)


folium_map = folium.Map(location=(32.2297700, 21.0117800), zoom_start=2)

for index, row in df.iterrows():
    put_piechart(folium_map, location=(row["lat"], row["lon"]), row=row)


Sample output stored in html - github is not supporting folium....

In [ ]:
folium_map.save("sample_demography.html")

Print out output on screen.

In [ ]:
folium_map